# Automatic Deep Research  - Adding coordination patterns

Welcome to the first practice lab of this module! 

In this lab, you will continue working on the deep research crew from Modules 1 and 2. This time you will be taking the crew one step further by adding coordination patterns.

**What you'll learn:**
- How to create a crew using the CrewAI CLI
- How to design agent teams using different coordination strategies

## Background

As a research consultant, you're constantly tasked with producing comprehensive reports on diverse topics for demanding clients. You need to build an AI research crew that can rapidly gather, verify, and synthesize information from across the internet, delivering reliable, fact-checked reports that meet tight deadlines and exacting standards regardless of the subject matter.

## General instructions
In this lab you will be presented with a structure of the code, but you will need to complete some of it. 

To successfully run this lab, replace all instances of the placeholder `None` with your own code. Sections where you need to write code will be delimited between `### START CODE HERE ###` and `### END CODE HERE ###`.

**<font color='#5DADEC'>Please make sure to save your work periodically, so you don't lose any progress.</font>**

## Table of contents

- [1. Problem statement](#1)

- [2. Project structure](#2)

- [3. Defining the tasks](#3)

- [4. Complete the guardrails file](#4)

- [5. Write the code to define the crew](#5)

- [6. Kickoff the crew](#6)


<a id="1"></a>

## 1. Problem statement

In the previous module you built a deep research crew that consisted of four tasks, with four agents: 

<img src="../images/m2-agents-tasks-diagram.PNG">

This time you will divide the research into main and secondary topics, and run the corresponding research and fact checking in parallel. The new structure should look something like this:

<img src="../images/m3-lab1-agents-tasks-diagram.png">

As you can see, although the research and data validation tasks have been split into main and secondary tracks, they both use the same agents, allowing you to reuse your existing agent definitions.

<a id="2"></a>

## 2. Project structure
In this lab, you will not only be learning how to add coordination patterns, you will also learn a new way to define crews, using CrewAI CLI. The CLI provides a set of commands to interact with CrewAI, allowing you to create, train, run, and manage crews. At the end of the day, it is just a different way of doing the things you've been doing so far, but it makes it easier to use your Crew in production later on.

### 2.1 Get familiar with the folder structure

Look at the left side panel of the jupyter lab. There you should see a folder structure similar to this one:
<div style="text-align: center;">
<img src='../images/lab1-folder-structure.png' width=200>
</div>

This structure was automatically created by running the command

```bash
crewai create crew deep_research_crew
```

This command creates a template for your crew project. If you navigate through the files in this lab, you will see that some of them have been already (partially) completed for you. 

Let's do a little walkthrough of the different files and folders:

- [`knowledge/user_preference.txt`](deep_research_crew/knowledge/user_preference.txt): This is one of the template files. Here you can add knowledge to your crew. You learned about knowledge on Module 2 of this course.
- [`src/deep_research_crew/config`](deep_research_crew/src/deep_research_crew/config/): This folder contains the YAML configuration files for agents and tasks. This is equivalent to the files you used in the previous module. You will be working on the `tasks.yaml` file later in the lab
- [`src/deep_research_crew/tools`](deep_research_crew/src/deep_research_crew/tools/): You should use this folder to define custom tools if needed. You can write your custom Python  code in the [`custom_tool.py`](deep_research_crew/src/deep_research_crew/tools/custom_tool.py) file, but you won't be working on this here.
- [`src/deep_research_crew/crew.py`](deep_research_crew/src/deep_research_crew/crew.py): In this file you will define the agents and tasks, and the crew. This is similar to what you have been doing in the previous labs. This file has been partially filled out for you, you will complete it later on in this lab. 

    Take close look at the code, and the use of decorators, which are statements starting with `@`. You will not need to modify them in this lab, but it is good that you start getting used to them. 
- [`src/deep_research_crew/main.py`](deep_research_crew/src/deep_research_crew/main.py): This file file is intended to be a way for your to run your crew locally. You can use it to test the crew by replacing with the desired inputs. This file is just the default generated by the `crewai create crew deep_research_crew` command, and you won't be making any changes to it.
- [`pyproject.toml`](deep_research_crew/pyproject.toml): This is a configuration file that defines build system requirements, dependencies, and metadata for the Python project, enabling standardized packaging and tool integration. As you can see, in this case there are some extra packages, which you need for the tools, both custom and built-in. You won't be making any changes to this file.

Take some time to go through each of these files so you get familiar with them.

<a id="3"></a>

## 3. Defining the tasks

Now it's time to define the tasks for this crew. Just as it happened with agents, tasks will be very similar to the ones in Module 2, but you need to add two additional tasks for the secondary topics.

Open the [src/deep_research_crew/config/tasks.yaml](deep_research_crew/src/deep_research_crew/config/tasks.yaml).

1. Complete the `agent` for each task. Remember that the main and secondary tasks share the same agent, so, for example, `research_main_topics` and `research_secondary_topics` share the same `topic_researcher` agent. 
2. Complete the `context` for each task. Remember that the context parameter passes the output of one task as input into another, creating a workflow where each task builds on the previous one. 

Look for the `### START CODE HERE ###` tags to know where to edit the files.

<a id="4"></a>

## 4. Complete the guardrails file

Now it's time to add the guardrail functions you defined during Module 2-Lab 1. 

Open the [src/deep_research_crew/guardrails/guardrails.py](deep_research_crew/src/deep_research_crew/guardrails/guardrails.py) file, and copy the following guardrail function.

```python
def write_report_guardrail(output):
    # get the raw output from the TaskOutput object
    try:
        output = output if type(output)==str else output.raw 
    except Exception as e:
        return (False, ("Error retrieving the `raw` argument: "
                        f"\n{str(e)}\n"
                        )
                )
    
    # convert the output to lowercase
    output_lower = output.lower()

    # check that the summary section exists
    if not re.search(r'#+.*summary', output_lower):
        return (False, 
                "The report must include a Summary section with a header like '## Summary'"
                )

    # check that the insights or recommendations sections exist
    if not re.search(r'#+.*insights|#+.*recommendations', output_lower):
        return (False, 
                "The report must include an Insights section with a header like '## Insights'"
                )

    # check that the citations (or references) section exists
    if not re.search(r'#+.*citations|#+.*references', output_lower):
        return (False, 
                "The report must include a Citations (or References) section with a header like '## Citations'"
                ) 
    return (True, output)
```

This is the same guardrail function you defined in the first practice lab in Module 2. 

<a id="5"></a>

## 5. Write the code to define the crew

Now it's time to define the crew using the [`src/deep_research_crew/crew.py`](deep_research_crew/src/deep_research_crew/crew.py) file. You will need to perform different actions on this file

1. Add the tools to the research agents. You need to add"
 - An instance of `EXASearchTool` and a `ScrapeWebsiteTool` for the **Interternet Researcher** and **Fact Checker** agents
 - An instance of the `ChartGeneratorTool` custom tool to the **Report Writer** agent
2. You want the main and secondary topics research to happen in parallel. For that to happen, you need to configure the `research_main_topics` and `research_secondary_topics` to run async (set the `async_execution` argument to `True`). 
3. When defining the **Write Final Report** task: 
 - Add the guardrail.
 - Enable Markdown output using the `markdown_output` argument.
 - Set the name of the output using the `output_file`.
 This last two steps replace the after kickoff execution hook you were using up to now. 
4. When defining the crew:
    - Set the process to `sequential`.
    - Allow the crew to use memory.

<a id="6"></a>

## 6. Kickoff the crew
Go to the [src/deep_research_crew/main.py](deep_research_crew/src/deep_research_crew/main.py) file and inside the `run()` function definition replace the `inputs` with your own query. You can use the same query you've been using so far.

After you are done, save the file and kickoff the Crew. Since you are working with the CrewAI CLI you actually need to run the code in terminal. There are two ways in which you can do this:

1. Run the cell below. If you are wondering what that code is doing, here is a breakdown:
    - `!`: tells the notebook to run that command in the system shell (terminal) rather than as Python code.
    - `cd deep_research_crew` changes the directory to the `deep_research_crew` folder
    - `crewai run` calls the CrewAI CLI and kicks off the crew

In [ ]:
# run this cell to avoid warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
! cd deep_research_crew && crewai run


2. Open a new terminal con copy these lines:
    ```bash
    cd M3/ugls/Lab1/deep_research_crew 
    crewai run
    ```
    **Note**: To open a terminal in a new tab you need to click on `File > New > Terminal`

    <div style="text-align: center;">
    <img src='../images/open_new_terminal.png' width=550>
    </div>


Congratulations on finishing this lab! We hope you found this new way of coding your crews exciting and fun.